# Havi Random

In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline  

### Optimization 
from scipy.optimize import brute
from scipy import optimize
import scipy.optimize as opt  

### Cluster 
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.metrics import silhouette_samples, silhouette_score


## Graph
import plotly.graph_objs as go
from plotly import tools
import plotly.plotly as py
import plotly
loca_path = './'

### Reading the DC Volume and Items data

In [3]:
DC_Volume = pd.read_excel(loca_path+"START HERE_DC Volumes with Case Attributes.xlsx")
items = pd.read_excel(loca_path+"START HERE_DC Volumes with Case Attributes.xlsx", sheetname=1)

In [4]:
items.head()

,Wrin Prefix,D,F,R,Rank
0,00004-FRENCH FRIES,NaN,2176482.0,NaN,1.0
1,00407-CHICKEN NUGGETS,NaN,757977.0,NaN,2.0
2,00005-10/1 BEEF PATTY,NaN,652402.0,NaN,3.0
3,00070-HASH BROWNS,NaN,648789.0,NaN,4.0
4,02813-READY MADE BISCUITS,NaN,438040.0,NaN,5.0


In [5]:
DC_Volume.head(1)


,Supplier Parent Name,Supplier WSI#,DC Receipts Cases Received,Wrin Prefix,Wrin Description,Wrin Temp Zone,WW Case Height,WW Case Length,WW Case Width,WW Weight UOM,WW Pallet TI x HI,WW Gross Weight (LBS),WW Case Dimensions (inches),DC ID
0,MCCAIN FOODS,47883,728707,00004-FRENCH FRIES,00004849-FRIES/440 NATURAL F16/6,F,13.5,16.38,13.13,LB,9.00 x 6.00,38.0,16.38 x 13.13 x 13.50,7


In [6]:
DC_Volume.columns

Index(['Supplier Parent Name', 'Supplier WSI#', 'DC Receipts Cases Received',
       'Wrin Prefix', 'Wrin Description', 'Wrin Temp Zone', 'WW Case Height',
       'WW Case Length', 'WW Case Width', 'WW Weight UOM', 'WW Pallet TI x HI',
       'WW Gross Weight (LBS)', 'WW Case Dimensions (inches)', 'DC ID'],
      dtype='object')

In [189]:
df_F = DC_Volume[DC_Volume['Wrin Temp Zone']=='F']

In [223]:
top_15_inF = items[items.F.notnull()][items.Rank.notnull()]['Wrin Prefix'];
df_F = df_F[df_F['Wrin Prefix'].isin(top_15_inF)];

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [224]:
df_F = DC_Volume[DC_Volume['Wrin Temp Zone']=='F']

# Cluster to pick the Size



In [50]:
def space_coef(input_size,*param):
    in_val = space_coef_input(input_size,*param)
    out_val  = space_coef_outbound(input_size,*param)
    return -(in_val**2 + out_val**2)**0.5

In [51]:
##  Maximum volume efficiency
#def space_coef(input_size,pallet ='O'):
def space_coef_input(input_size,*param):
    ## Define the pallet
    height,length,width = input_size
    pallet ='O'
#     if params:
#         pallet = params
#     else:
#         pallet = 'O'
        
    coef = 0
    if pallet == 'O':# height ,weight,length for pallet
        Hp = 100
        Lp = 48
        Wp = 40
    else:
        Hp = 100
        Lp = 42
        Wp = 29
    vol_case = height*length*width
    num_H = np.floor(Hp/height)
    num_L = np.floor(Lp/length)
    num_W = np.floor(Wp/width)
    coef = num_H*num_L*num_W*(vol_case)/(Hp*Lp*Wp)
    # since all the optimization function I can find so far is about to find minimum value, 
    # that's the most easy way to get the value right...
    return -coef
    

In [52]:
##  Maximum volume efficiency
#def space_coef(input_size,pallet ='O'):
def space_coef_outbound(input_size,*param):
    ## Define the pallet
    height,length,width = input_size
    pallet ='I'
#     if params:
#         pallet = params
#     else:
#         pallet = 'O'
        
    coef = 0
    if pallet == 'O':# height ,weight,length for pallet
        Hp = 100
        Lp = 48
        Wp = 40
    else:
        Hp = 100
        Lp = 42
        Wp = 29
    vol_case = height*length*width
    num_H = np.floor(Hp/height)
    num_L = np.floor(Lp/length)
    num_W = np.floor(Wp/width)
    coef = num_H*num_L*num_W*(vol_case)/(Hp*Lp*Wp)
    # since all the optimization function I can find so far is about to find minimum value, 
    # that's the most easy way to get the value right...
    return -coef

In [53]:
#brief create interval for the matrix, want to use 0.2 inch as an unit.
#input mat it should be np.array, n x 3 matrix
#input
#Return:   2 element list, lower bound,  upeper bound
def UpLowbound_gen(mat,limit = 'vol'):
    pect_ = 1.1 # 10% increase or decreaase
    unit = 1/0.2
    if limit =='vol':
        pect_ = pect_ ** (1. / 3)

    else:
        pect_ = pect_
        
    upper_mat = np.multiply(mat,pect_)
    upper_mat = np.divide(np.floor(np.multiply(upper_mat,unit)),unit)
    lower_mat = np.around(np.divide(mat,pect_),decimals=2)
    lower_mat = np.divide(np.ceil(np.multiply(lower_mat,unit)),unit)
    return lower_mat, upper_mat,np.dstack([lower_mat,upper_mat])

### Kmeans Cluster  Create

(Not yet done)New Idea to eliminate outlier
-Using  Guassian Mixture clsuter
=> K-means to get new cluster

In [147]:
X_train = df_F.loc[:,['WW Case Height','WW Case Length', 'WW Case Width']]

In [14]:
kmean_estimators  = KMeans(n_clusters=8)
kmean_estimators.fit(X_train)
labels = kmean_estimators.labels_

elbow method for getting the number of clusters

### silhouette_score from 5 to 15
~~~ modified by grid search. later.

In [151]:
kmean_estimators  = KMeans(n_clusters=33)
kmean_estimators.fit(X_train)
labels = kmean_estimators.labels_
score = silhouette_score(X_train, labels, metric='euclidean')

In [150]:
for i in range(10,60):
    kmean_estimators  = KMeans(n_clusters=i)
    kmean_estimators.fit(X_train)
    labels = kmean_estimators.labels_
    score = silhouette_score(X_train, labels, metric='euclidean')
    ls_sil.append(score)
    print(score)

0.614259563694
0.623763152903
0.644270381707
0.561698363615
0.577124367936
0.591542554669
0.57137105004
0.595512858078
0.605628758812
0.620417015502
0.643740902713
0.661535752975
0.659149884382
0.665375105785
0.650084727658
0.686848776747
0.679601256236
0.679626798765
0.68367175304
0.692304151571
0.679909085097
0.688541483628
0.710388135467
0.707700419742
0.69620174333
0.695498735313
0.698185257036
0.696557943668
0.709308357668
0.707661996236
0.70242448605
0.7076564656
0.692656940564
0.688385773619
0.697809896648
0.687918423137
0.6915955792
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106
0.698795063106


### Generate Case size bound

In [226]:
## to get the H,L,W range. 
kmean_estimators.cluster_centers_= X_train.iloc[:,0:3] ## Put the all data into optmize
Lmat,Umat ,ULmat = UpLowbound_gen(kmean_estimators.cluster_centers_)
ideal_input_size = list()
for limit in ULmat:
    H_MinMax, L_MinMax, W_MinMax = limit
    rranges = (slice(H_MinMax[0],H_MinMax[1], 0.2), slice(L_MinMax[0],L_MinMax[1], 0.2)
           ,slice(W_MinMax[0],W_MinMax[1], 0.2))
    resbrute = brute(space_coef_input, rranges, full_output=True, finish=None)
    ideal_input_size.append(resbrute[0])## ideal size is correspond to original cluster center.

In [227]:
## to get the H,L,W range. 
Lmat,Umat ,ULmat = UpLowbound_gen(kmean_estimators.cluster_centers_)
ideal_size = list()
for limit in ULmat:
    H_MinMax, L_MinMax, W_MinMax = limit
    rranges = (slice(H_MinMax[0],H_MinMax[1], 0.2), slice(L_MinMax[0],L_MinMax[1], 0.2)
           ,slice(W_MinMax[0],W_MinMax[1], 0.2))
    resbrute = brute(space_coef, rranges, full_output=True, finish=None)
    ideal_size.append(resbrute[0])## ideal size is correspond to original cluster center.

In [187]:
#Compute difference Optimization result from two differnent Object function
diff_counter = 0;
for i in range(len(in_size)):
    if any(ideal_input_size[i] != ideal_size[i]):
        diff_counter+=1

### Pick the best K from silhouette_score

k = 

In [217]:
k = 9
kmean_estimators  = KMeans(n_clusters=k)
kmean_estimators.fit(X_train)
labels = kmean_estimators.labels_

In [218]:
X_train['class'] = labels

### Plot 3-D graph

In [ ]:
def color_gen(num):
    ls_color =list()
    for i in range(num):
        ls_color.append("rgb({0:d},{1:d},{2:d})".format(int(np.random.uniform(128,255,1)),int(np.random.uniform(0,255,1)),int(np.random.uniform(0,255,1))))
    return ls_color

In [220]:
data = []
clusters = []
colors = color_gen(k)
for i in range(k):
    color =  colors[i]
    dx = X_train[X_train['class'] == i].iloc[:, 0],
    dy = X_train[X_train['class'] == i].iloc[:,  1],
    dz = X_train[X_train['class'] == i].iloc[:,  2],
    x = dx[0]
    y = dy[0]
    z = dz[0]
    print(x)
    trace = go.Scatter3d(
        x = x, y = y, z = z,   
        mode = 'markers',
        marker = dict( size=10, color=color, line=dict(width=0) ) )
    data.append( trace )
    
#     cluster = dict(
#         color = color,
#         opacity = 0.3,
#         type = "mesh3d",    
#         x = x, y = y, z = z )
#     data.append( cluster )


layout = go.Layout(
    title='Plot Title',
    xaxis=dict(
        title='x Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='y Axis',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='simple-3d-scatter.html')

68     8.49
69     8.00
70     8.25
71     8.25
248    9.00
249    9.41
318    7.50
319    7.50
320    7.50
Name: WW Case Height, dtype: float64
0      13.50
1      12.38
2      12.38
3      13.75
4      12.38
72     12.25
73     12.25
74     12.25
75     12.38
143    12.81
144    12.19
145    12.81
201    12.50
202    13.81
203    13.81
204    13.81
215    12.94
227    12.82
228    12.82
229    12.82
272    12.38
273    12.38
274    15.69
304    13.00
305    13.00
306    13.00
573    13.00
574    13.00
575    13.00
Name: WW Case Height, dtype: float64
257    10.00
258    10.00
259    10.00
296     9.00
297     9.00
298     9.00
299     7.19
381     9.40
382     9.40
383    10.00
384     9.40
Name: WW Case Height, dtype: float64
8      7.81
9      8.25
10     8.25
152    7.38
153    8.50
154    8.00
Name: WW Case Height, dtype: float64
287    13.63
288    12.50
289    13.38
Name: WW Case Height, dtype: float64
13     10.63
14     10.63
15     11.25
246    11.13
247    11.13
Name: WW Ca

'file://C:\\Users\\csblo\\Documents\\Math565\\HAVI\\RD_HAVI\\simple-3d-scatter.html'

In [211]:
print(x[0])

5       8.00
7       8.00
8       7.81
9       8.25
10      8.25
68      8.49
69      8.00
70      8.25
71      8.25
152     7.38
153     8.50
154     8.00
158    10.00
159     9.88
160     9.88
198     5.94
199     5.94
200     5.94
248     9.00
249     9.41
318     7.50
319     7.50
320     7.50
Name: WW Case Height, dtype: float64


## Understanding the data

### Mismatch of items

In comparing the unique "WRIN PREFIX", the following items are missing from "DC Volumes" and "Xtab-Temp Zone X WRIN" sheets.
1. 00042-DIET COKE SYRUP - with 152,368 cases received
2. 00168-SPRITE DRINK SYRUP - with 142,032 cases received

Both the WRIN PREFIX has temperature zone as Dry with the same Supplier "COCA-COLA COMPANY (THE)/MCDONALDS ACCOUNT GROUP". Adding these two tiems in the "Xtab-Temp Zone X WRIN" sheet will change the rank of other items.

In [ ]:
temperature = ('F', 'R', 'D')
for i in temperature:
    print("Temp zone", i ,"missing:",set(DC_Volume['Wrin Prefix'][DC_Volume['Wrin Temp Zone'] == i].unique()).symmetric_difference(set(items['Wrin Prefix'][items[i].notnull()].unique())))

In [ ]:
print("Total number of items manufactured and distributed:",len(DC_Volume['Wrin Prefix'].unique()), "items")
print("Number of DCs:", DC_Volume['DC ID'].unique())
print("Total number of suppliers:",len(DC_Volume['Supplier Parent Name'].unique()))
print("Freezer items:",len(DC_Volume['Wrin Prefix'][DC_Volume['Wrin Temp Zone'] == "F"].unique()))
print("Dry items:",len(DC_Volume['Wrin Prefix'][DC_Volume['Wrin Temp Zone'] == "D"].unique()))
print("Refrigerator items:",len(DC_Volume['Wrin Prefix'][DC_Volume['Wrin Temp Zone'] == "R"].unique()))

### Wrong Temperature Zone

"LOPEZ FOOD INC" is the only Supplier to pack Beef patty in Temp Zone "R" and sent 105 cases in total. They are also responsible for sending "Canadian Bacon/Natural juice" in Temp Zone "R"

In [ ]:
DC_Volume[(DC_Volume['Wrin Prefix']=='00006-4/1 BEEF PATTY') & (DC_Volume['Wrin Temp Zone']=='R')]

Find the most number of cases sent by a supplier

Why is 1 item measured in KG but the gross weight in LBS. is this typo error?

Check what all items goes to each DCs

### Basic visualization

In [ ]:
#Top and least suppliers for Mcdonalds
#plt.figure()

plt.figure(figsize=(50,20))
plt.subplot(121)

df = DC_Volume[['Supplier Parent Name','DC Receipts Cases Received']]
df['Supplier Parent Name']=df['Supplier Parent Name'].replace(['COCA-COLA COMPANY (THE)/MCDONALDS ACCOUNT GROUP'],value='COCA-COLA MCD')
df['Supplier Parent Name']=df['Supplier Parent Name'].replace(['COCA-COLA COMPANY (THE)/MOTHER PARKERS FOODS LTD'],value='COCA-COLA Mother')
grouped = df.groupby(df['Supplier Parent Name']).sum().reset_index()
grouped = grouped.sort_values('DC Receipts Cases Received', ascending = False)
sns.barplot(grouped['Supplier Parent Name'][:15], grouped['DC Receipts Cases Received'][:15])
plt.title("Top 15 Suppliers",size=50)
plt.ylabel("Cases Recevied by DC",size=40)
plt.xticks(rotation=90,size=30)
plt.yticks(size=30)
plt.xlabel('Supplier Name',size=50)

plt.subplot(122)
sns.barplot(grouped['Supplier Parent Name'][-15:], grouped['DC Receipts Cases Received'][-15:])
plt.title("Least 15 Suppliers",size=50)
plt.ylabel("Cases Recevied by DC",size=40)
plt.xticks(rotation=90,size=30)
plt.yticks(size=30)
plt.xlabel('Supplier Name',size=50)
plt.show()


# DC outbound

In [ ]:
df_outbound = pd.read_excel(loca_path+"DC Sales_DC7.xlsx")

In [ ]:
df_outbound.head(1)

In [ ]:
len(df_outbound['Rest ID'].unique())